In [1]:
import pandas as pd
import sys
import importlib
import fasttext
import numpy as np
import tensorflow as tf

base_dir = '/Users/anasputhawala/Desktop/Winterproj/'
sys.path.insert(0, base_dir)

from src.utils import pre_processing
from src.models.tfidf import config_tfidf
import fasttext_prep
importlib.reload(pre_processing)
importlib.reload(config_tfidf)
importlib.reload(fasttext_prep)

2022-12-27 15:59:32.405076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 15:59:39.493313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<module 'fasttext_prep' from '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/fasttext_prep.py'>

In [10]:
fasttext_prep.load_prep_save()

Train and Test .txt files have successfully been saved at /Users/anasputhawala/Desktop/Winterproj/src/models/fasttext


In [41]:
train_path = '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/train.txt'
test_path = '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/test.txt'
model_ft = fasttext.train_supervised(input=train_path, wordNgrams=3, epoch=5, lr=0.15)

Read 1M words
Number of words:  88486
Number of labels: 16
Progress: 100.0% words/sec/thread:  910919 lr:  0.000000 avg.loss:  2.199809 ETA:   0h 0m 0s


In [42]:
model_ft.test(test_path)

(54420, 0.2984196986402058, 0.2984196986402058)

Using a different approach (gensim library)

In [370]:
from gensim.models import FastText
from gensim.utils import simple_preprocess

import ft_model
importlib.reload(ft_model)
importlib.reload(fasttext_prep)

<module 'fasttext_prep' from '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/fasttext_prep.py'>

In [373]:
model = ft_model.FastTextModel(model_dir)
model.build(input_shape=X_train.shape)

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, call your model on real tensor data (of the correct dtype).

The actual error from `call` is: Exception encountered when calling layer 'fast_text_embedding_82' (type FastTextEmbedding).

in user code:

    File "/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/ft_model.py", line 26, in call  *
        sent_tokenized = simple_preprocess(input)
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/gensim/utils.py", line 310, in simple_preprocess  *
        tokens = [
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/gensim/utils.py", line 262, in tokenize  *
        text = to_unicode(text, encoding, errors=errors)
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/gensim/utils.py", line 365, in any2unicode  *
      

In [321]:
# df = fasttext_prep.load_gensim()
# (X_train, y_train), (X_val, y_val), (X_test, y_test) = fasttext_prep.prep_gensim(df, tokenize=False, train_ratio=0.75, val_ratio=0.15, test_ratio=0.1)

Using lambda layer

In [372]:
LR=0.001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

history = model.fit(X_train,
                    y_train,
                    batch_size=128,
                    validation_data=(X_val, y_val),
                    validation_batch_size=128,
                    epochs=15)

Epoch 1/15


TypeError: in user code:

    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/anasputhawala/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File 

In [319]:
model(["Hey my name is Anas"])

TypeError: Exception encountered when calling layer 'fast_text_embedding_46' (type FastTextEmbedding).

decoding to str: need a bytes-like object, list found

Call arguments received by layer 'fast_text_embedding_46' (type FastTextEmbedding):
  • input=["'Hey my name is Anas'"]

In [293]:
m = FastText.load(model_dir)
def p(text):
    sent_tokenized = text.lower().split(' ')
    out = np.zeros(shape=(60,))
    # sent_tokenized = simple_preprocess(input)

    # we'll use mean pooling
    for word in sent_tokenized:
        out[:] += m.wv[word]
    
    return tf.convert_to_tensor(out / len(sent_tokenized))

In [300]:
test_vec = tf.keras.layers.TextVectorization(
            max_tokens=20000,
            standardize='lower_and_strip_punctuation',
            split='whitespace',
            ngrams=None,
            output_mode='int',
            output_sequence_length = 12)

In [302]:
test_vec.adapt(X_train)

In [306]:
test_vec.

Signature:      test_vec(*args, **kwargs)
Type:           TextVectorization
String form:    <keras.layers.preprocessing.text_vectorization.TextVectorization object at 0x169ec3fa0>
File:           ~/Desktop/Winterproj/emojify/lib/python3.9/site-packages/keras/layers/preprocessing/text_vectorization.py
Source:        
@keras_export(
    "keras.layers.TextVectorization",
    "keras.layers.experimental.preprocessing.TextVectorization",
    v1=[],
)
class TextVectorization(base_preprocessing_layer.PreprocessingLayer):
    """A preprocessing layer which maps text features to integer sequences.

    This layer has basic options for managing text in a Keras model. It
    transforms a batch of strings (one example = one string) into either a list
    of token indices (one example = 1D tensor of integer token indices) or a
    dense representation (one example = 1D tensor of float values representing
    data about the example's tokens). This layer is meant to handle natural
    language inputs.

In [294]:
# ft_embedding = ft_model.FastTextEmbedding(trained_ft_model_dir=model_dir)
# ft_embedding.build(input_shape=[])

inp = tf.keras.Input(shape=X_train.shape, dtype="string")
# embedding = ft_embedding(inp)
embedding = tf.keras.layers.Lambda(lambda z: p(z))(inp)
dense = tf.keras.layers.Dense(units=16, activation='softmax')(embedding)

m2 = tf.keras.Model(inputs=inp, outputs=dense)


AttributeError: Exception encountered when calling layer "lambda_6" (type Lambda).

'Tensor' object has no attribute 'lower'

Call arguments received by layer "lambda_6" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 204073), dtype=string)
  • mask=None
  • training=None

In [160]:
print(f'Training data: {X_train}\n\nshape of training data: {X_train.shape}')
print('\n')
print(f'Labels for data: {y_train}\n\nshape of labels for training: {y_train.shape}')

Training data: ['chick fil hicksville finally open'
 'accidental twinning with my favorites'
 'one of my favorite people in this world love you mrs andrews' ...
 'of your fav artists takin on hoco' 'welcome to the studio' 'the boys']

shape of training data: (204073,)


Labels for data: [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

shape of labels for training: (204073, 16)
